In [13]:
from utils.molecule_tools import construct_molecule
from utils.cs_vqe_tools_original import greedy_dfs
from cs_vqe.circuit import cs_vqe_circuit
import json

In [14]:
with open('data/molecule_data.json', 'r') as json_file:
    molecule_data = json.load(json_file)
with open('data/model_data.json', 'r') as json_file:
    model_data = json.load(json_file)
#with open('data/anz_circ_depth.json', 'r') as json_file:
#    anz_data = json.load(json_file)
    
#speciesname_list = [mol for mol in anz_data if (anz_data[mol]['num_qubits']!=anz_data[mol]['chemaccnum'] and not anz_data[mol]['chemaccnum']==0) and mol.find('alt')==-1 and mol.find('+')==-1]
#speciesname_list

In [18]:
#model_data = {}

for speciesname in ['HCl_STO-3G_SINGLET']:#speciesname_list:
    print(speciesname)
    
    atoms, coords, multiplicity, charge, basis, sym_sector = molecule_data[speciesname].values()
    molecule    = construct_molecule(atoms, coords, charge, multiplicity, basis, 
                                     excitation_threshold=5, taper=True, sym_sector=sym_sector)
    num_qubits  = molecule['num_qubits']
    ham         = molecule['hamiltonian']
    ansatze     = molecule['ansatze']
    hf_config   = molecule['hf_config']
    hf_energy   = molecule['hf_energy']
    num_tapered = molecule['num_tapered']
    true_gs_nrg = molecule['true_gs_nrg']
    #true_gs_vec = molecule['true_gs_vec']

    terms_noncon = greedy_dfs(ham, 30, criterion='size')[-1]

    print('Number of qubits:', num_qubits)
    mol_circ = cs_vqe_circuit(hamiltonian = ham,
                              terms_noncon= terms_noncon,
                              num_qubits  = num_qubits,
                              hf_config   = hf_config)

    ham_rotations  = mol_circ.ham_rotations
    noncon         = mol_circ.noncon
    truegs         = mol_circ.truegs
    G              = mol_circ.G
    A              = mol_circ.A
    X_index        = mol_circ.X_index
    X_qubit        = mol_circ.X_qubit
    cs_vqe_energy  = mol_circ.cs_vqe_energy
    cs_vqe_errors  = mol_circ.cs_vqe_errors
    chem_acc_num_q = mol_circ.chem_acc_num_q
    order          = mol_circ.order
    ham_reduced    = mol_circ.ham_reduced
    reference_state= mol_circ.reference_state()

    model_data[speciesname] = {"ham":ham,
                               "ansatze":ansatze,
                               "num_qubits":num_qubits,
                               "hf_config":hf_config,
                               "hf_energy":hf_energy,
                               "terms_noncon":terms_noncon,
                               "num_tapered":num_tapered,
                               "true_gs_nrg":true_gs_nrg,
                               #"true_gs_vec":true_gs_vec,
                               "ham_rotations":ham_rotations,
                               "noncon":noncon,
                               "truegs":truegs,
                               "G":G,
                               "A":A,
                               "X_index":X_index,
                               "X_qubit":X_qubit,
                               "cs_vqe_energy":cs_vqe_energy,
                               "cs_vqe_errors":cs_vqe_errors,
                               "chem_acc_num_q":chem_acc_num_q,
                               "order":order,
                               "ham_reduced":ham_reduced,
                               "reference_state":reference_state
                                }

    print('Molecule constructed.\n')


HCl_STO-3G_SINGLET


ValueError: 'to_matrix' will return an exponentially large matrix, in this case '1048576x1048576' elements. Set algorithm_globals.massive=True or the method argument massive=True if you want to proceed.

In [6]:
#data={}
#for k in model_data.keys():
#    data[str(k)] = model_data[k]
     
with open("data/model_data.json", "w") as outfile:
    json.dump(model_data, outfile)

In [12]:
for name, a in model_data['HF_STO-3G_SINGLET']['ansatze'].items():
    print(name, len(a))

uccsd 44
excite3 8
excite4 44
excite5 44


In [20]:
model_data.keys()

dict_keys(['H2O_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'CH2_STO-3G_SINGLET', 'BeH2_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'C_STO-3G_SINGLET', 'NH_STO-3G_SINGLET', 'BH_STO-3G_SINGLET'])